<a href="https://colab.research.google.com/github/aedhcarrick/nacho-diffusion/blob/main/nacho_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title -=Setup =-
!nvidia-smi -L
%cd /content
!echo -= Download Repo =-
!git clone https://github.com/aedhcarrick/nacho-diffusion
%cd /content/nacho-diffusion
!git checkout main
!git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

!echo -= Download Model =-
!wget -c https://huggingface.co/XpucT/Deliberate/resolve/main/deliberate_v2.ckpt -O ./models/checkpoints/model.ckpt


In [ ]:
#@title -= Start Worker =-
import random
from image_worker.image_worker import ImageWorker

input_dir = 'input'
output_dir = 'output'
model_dir = 'models'

worker = ImageWorker(input_dir, output_dir, model_dir)
worker.start()

In [ ]:
#@title -= Nacho Generator =-

PROMPT = 'A painting of a dog eating nachos, style of Caravaggio.' #@param {type:"string"}
SAMPLER = "DDIM" #@param ["DDIM", "DPM", "PLMS"]
STEPS = 50 #@param {type:"slider", min:1, max:100, step:1}
GUIDANCE = 7.5 #@param {type:"slider", min:1, max:20, step:0.5}
SEED = 42 #@param {type:"integer"}
HEIGHT = 512 #@param {type:"slider", min:"512", max:"1024", step:64}
WIDTH = 512 #@param {type:"slider", min:"512", max:"1024", step:64}
BATCH_SIZE = 1 #@param {type:"integer"}

if SEED < 1:
    SEED = random.randint(1, 1000000000)

job = {
    "job_type": "image",
    "operations": [
        {
        "oper_type": "txt2img",
        "prompt": PROMPT,
        "sampler": SAMPLER,
        "steps": STEPS,
        "height": HEIGHT,
        "width": WIDTH,
        "guidance": GUIDANCE,
        "seed": SEED,
        "batch_size": BATCH_SIZE,
        }
    ]
}


worker.submit_job('txt2img', job)


In [ ]:
#@title -= Stop Worker =-
worker.stop()

In [ ]:
#@title -= Pytest =-
!python -m pytest